## 분석 방향(직원수)

### 값을 기준으로 소거
- 예비유니콘 선정 된 기업 중 가장 적은 직원 수의 기업이 47명 (디스이즈엔지니어링) (출처 : 중소벤처기업부)
- 유니콘 선정 기업 중 가장 적은 직원 수는 121명


### 변화율을 기준으로 소거
- 찾아놨던 유니콘 기업들의 데이터에서 연평균 직원수 성장률을 분석 후 평균값을 기준으로 사용
- 주어진 데이터 안에서 기준에 맞는 회사 추출

In [1]:
import pandas as pd
import numpy as np
import math

unicon_data = pd.read_csv("유니콘 취합.csv", index_col=0)
data = pd.read_csv("../data/company_nps_data.csv",index_col=0)
pd.options.display.max_rows = 60

## 연평균 성장률을 구하기 위한 함수

In [2]:
def cagr(start_value, end_value, num_periods):
    return (end_value / start_value) ** (1 / (num_periods - 1)) - 1

## 유니콘 회사의 직원수 데이터를 연도별로 재정렬

In [3]:
유니콘회사_연도별_직원 = unicon_data.groupby(['회사ID','년도'])['월별_직원수'].sum()
유니콘회사_연도별_직원 = 유니콘회사_연도별_직원.unstack()
유니콘회사_연도별_직원

년도,2015,2016,2017,2018,2019
회사ID,,,,,
무신사,127,891,1112,1723,3198
쏘카,178,2095,3386,3322,3995
야놀자,267,2536,3106,4193,7442
에이프로젠,169,1197,1677,1586,1890
엘앤피코스메틱,192,1895,2438,2643,2896
옐로모바일,70,459,1806,759,211
우아한형제들,437,3472,5463,7124,10175
지피클럽,15,120,147,369,1186
쿠팡,11003,62145,64672,69548,91767


## 유니콘 기업의 데이터를 바탕으로 연평균 성장률의 최솟값 구하기

In [4]:
유니콘기업_직원_성장률 = []
for company in range(0, 11):
   유니콘기업_직원_성장률.append(cagr(유니콘회사_연도별_직원.iloc[company, 0], 유니콘회사_연도별_직원.iloc[company, -1], 5))
유니콘기업_최소_성장률 = np.min(유니콘기업_직원_성장률)
유니콘기업_최소_성장률

0.3176379776996776

In [5]:
data

,연매출액,년도,월,월별_연금보험료,월별_직원수
회사ID,,,,,
233757,41688077,2018,7,8912380,36
123464,1760695,2018,3,942380,5
342124,3221341,2018,7,781180,4
386420,4815584,2015,11,3795900,14
129243,9799736,2018,10,40724680,151
...,...,...,...,...,...
171439,4630622,2017,2,2402740,11
239388,1045021,2018,10,1428800,9
272304,181898,2018,3,392560,4


## 주어진 데이터의 회사들을 연도별로 재정렬

In [6]:
연도별_직원 = data.pivot_table('월별_직원수', '회사ID', '년도')
연도별_직원.index = 연도별_직원.index.map(lambda x : str(x))
연도별_직원.columns = 연도별_직원.columns.map(lambda x : str(x))
연도별_직원

년도,2015,2016,2017,2018,2019
회사ID,,,,,
63,NaN,NaN,NaN,1007.166667,NaN
95,NaN,NaN,NaN,541.000000,NaN
102,NaN,NaN,NaN,752.125000,NaN
295,NaN,NaN,NaN,259.500000,NaN
414,NaN,NaN,NaN,247.583333,NaN
...,...,...,...,...,...
2400284,NaN,NaN,NaN,4.714286,NaN
2437574,NaN,NaN,NaN,3.000000,NaN
2693684,NaN,NaN,NaN,NaN,3.0


## 데이터가 존재하는 연도 1개 이하인 기업 삭제하기

In [7]:
# 연도별_직원['기간'] = 0
# for i in range(len(연도별_직원)):
#     연도별_직원['기간'][i] = 연도별_직원.iloc[i].notnull().sum()
# 연도별_직원['기간'] = 연도별_직원['기간'] - 2
# zero_duration = 연도별_직원[연도별_직원['기간'] == 0].index
# 연도별_직원 = 연도별_직원.drop(zero_duration)
# 연도별_직원

# NaN이 아닌 값(정상적인 데이터)이 2개 미만인 행 제거
연도별_직원.dropna(thresh=2, inplace=True)
id = 연도별_직원.index
연도별_직원

년도,2015,2016,2017,2018,2019
회사ID,,,,,
14271,13.0,14.916667,16.750000,20.166667,22.416667
44851,NaN,6.583333,NaN,4.000000,NaN
59533,5.0,5.916667,5.166667,5.500000,NaN
122968,88.0,76.166667,71.250000,66.500000,NaN
122983,63.5,58.916667,38.250000,NaN,NaN
...,...,...,...,...,...
515562,3.0,3.000000,NaN,NaN,NaN
515837,4.5,4.000000,NaN,NaN,NaN
516607,NaN,3.333333,NaN,3.000000,NaN


## 기업의 데이터에서 연평균 성장률의 평균값 구하기

In [8]:
직원_성장률 = {}
for company in id:
    start = 0
    end = 0
    count = 0
    for i in range(0, 5):
        if math.isnan(연도별_직원.loc[company][i]):
            continue
        else:
            if start == 0:
                start = i
            end = i
            count += 1
    직원_성장률[company] = cagr(연도별_직원.loc[company][start], 연도별_직원.loc[company][end], count)
직원수_성장률_분류 = dict(filter(lambda elem:elem[1]>=0.32, 직원_성장률.items()))
직원수_성장률_분류 = pd.Series(직원수_성장률_분류)

## 주어진 데이터에서 직원수가 47명 이상인 기업 선별

In [9]:
직원수_분류 = data.query("월별_직원수 > 46")
직원수_분류 = 직원수_분류.pivot_table('월별_직원수', '회사ID', '년도')
직원수_분류.index = 직원수_분류.index.map(lambda x : str(x))
직원수_분류.columns = 직원수_분류.columns.map(lambda x : str(x))
직원수_분류 = 직원수_분류.mean(axis=1)
직원수_분류

회사ID
63        1007.166667
95         541.000000
102        752.125000
295        259.500000
414        247.583333
             ...     
613233     113.500000
613313      64.750000
613430      48.750000
630190     821.666667
635082     206.833333
Length: 573, dtype: float64

## 직원수가 47명 이상이고 최소 연평균 성장률이 0.32 이상인 회사 검색

In [17]:
result = {}
for company in 직원수_성장률_분류.index:
    if company in 직원수_분류.index:
        result[company] = 직원수_성장률_분류[company]
    else:
        continue
result

{'129633': 1.0692144734142923,
 '131842': 0.6909409916155409,
 '134911': 0.7031992205741073,
 '135158': 0.4640166357994744,
 '137755': 0.6951944462291977,
 '296723': 0.9600665525005927,
 '297175': 0.3213711756350046,
 '297402': 0.7227884373886424,
 '380843': 8.19652650822669,
 '394749': 0.9371657742122697,
 '403470': 0.38497875787389324,
 '420008': 0.46530169282235034,
 '441997': 0.44143626841461847,
 '470994': 2.723797345005051,
 '510449': 0.5088944303988843}

## 최종결과
### 모든 조건에 들어맞는 기업은 24개

In [18]:
result_rate = pd.Series(result)
result_rate.sort_values(ascending = False, inplace=True)

### 직원 수의 평균이 높은 대로 출력

In [12]:
result_value = {}
for company in result.keys():
    if company in 직원수_분류.index:
        result_value[company] = 직원수_분류[company]
result_value = pd.Series(result_value)
result_value.sort_values(ascending = False, inplace=True)

In [13]:
writer = pd.ExcelWriter('result.xlsx')

result_value.to_excel(writer, sheet_name='직원수')
result_rate.to_excel(writer, sheet_name='직원수_변동률')
writer.close()